# Modeling to predict Circuit District


Revathi's tasks:
redo yohannes EDA with our csv
do 2 different logistic regressions on ERIOD and SENTIMP as targets
research on time periods
1 = Koon Period (June 13, 1996 through April 30, 2003)
2 = PROTECT Act Period (May 1, 2003 through June 24, 2004)
3 = Booker Period (January 12, 2005 through December 10, 2007)
4 = Gall Period (December 11, 2007 through September 30, 2011)


In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from skopt import BayesSearchCV
import pickle

In [2]:
#reading in file 
booker = pd.read_csv('../data/clean_booker.csv')
booker = booker.iloc[:, 1:]
booker

,AGE,GGDUM,BOOKER2,CAREER,AROFFAP,CIRCDIST,IMPACTOL,MPCTCHC,MITDUM,MONCIRC,...,SENTIMP,OTCHPTS,WEAPON,XCRHISSR,LMIN,FY,USSCIDN,ENSPLT0,LOSS_2B,GDL
0,28.0,0.0,2.0,0.0,0.0,80.0,-999.0,-999.0,0.0,10.0,...,1.0,5.0,0.0,3.0,10.0,2005.0,890751.0,8.0,-999.0,2L1.2
1,24.0,0.0,2.0,0.0,0.0,80.0,-999.0,-999.0,0.0,10.0,...,1.0,3.0,0.0,2.0,168.0,2005.0,890752.0,72.0,-999.0,2A3.1
2,32.0,0.0,0.0,0.0,0.0,33.0,-999.0,-999.0,0.0,5.0,...,1.0,3.0,0.0,2.0,12.0,2005.0,890755.0,15.0,-999.0,2D1.1
3,24.0,1.0,0.0,0.0,0.0,33.0,-999.0,-999.0,0.0,5.0,...,1.0,10.0,1.0,5.0,228.0,2005.0,890756.0,270.0,-999.0,2D1.1
4,43.0,0.0,0.0,0.0,0.0,93.0,-999.0,-999.0,0.0,11.0,...,1.0,9.0,0.0,4.0,57.0,2005.0,890757.0,60.0,-999.0,2L1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,40.0,0.0,2.0,0.0,0.0,47.0,-999.0,-999.0,0.0,7.0,...,4.0,0.0,0.0,1.0,6.0,2004.0,759721.0,0.0,33475.0,2BNew
799996,41.0,0.0,0.0,0.0,0.0,47.0,-999.0,-999.0,0.0,7.0,...,1.0,0.0,0.0,1.0,24.0,2004.0,759722.0,24.0,-999.0,2L2.1
799997,23.0,0.0,0.0,0.0,0.0,47.0,-999.0,-999.0,0.0,7.0,...,3.0,1.0,0.0,1.0,6.0,2004.0,759723.0,6.0,-999.0,2B1.1
799998,30.0,0.0,0.0,0.0,0.0,47.0,-999.0,-999.0,0.0,7.0,...,1.0,8.0,0.0,4.0,57.0,2004.0,759724.0,61.0,-999.0,2L1.2


In [3]:
booker.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 31 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   AGE        800000 non-null  float64
 1   GGDUM      800000 non-null  float64
 2   BOOKER2    800000 non-null  float64
 3   CAREER     800000 non-null  float64
 4   AROFFAP    800000 non-null  float64
 5   CIRCDIST   800000 non-null  float64
 6   IMPACTOL   800000 non-null  float64
 7   MPCTCHC    800000 non-null  float64
 8   MITDUM     800000 non-null  float64
 9   MONCIRC    800000 non-null  float64
 10  ONSEX      800000 non-null  float64
 11  NEWCIT     800000 non-null  float64
 12  NEWCNVTN   800000 non-null  float64
 13  EWEDUC     800000 non-null  float64
 14  NEWRACE    800000 non-null  float64
 15  OFFTYPE2   800000 non-null  float64
 16  ERIOD      800000 non-null  float64
 17  PRIMARY    800000 non-null  float64
 18  QUARTER    800000 non-null  float64
 19  A          800000 non-n

In [4]:
# defining X and y and splitting into traintest sets
y = booker['CIRCDIST']
X = booker.drop(columns = ['CIRCDIST'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, random_state=42)

NOTE: ENSPLT0 feature was not included in training because of its high correlation to the target variable, SENTIMP. ENSPLT0 indicates the length of the individuals' sentence, so it had high determination for SENTIMP targets 1, 2 and 3.

In [5]:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

ValueError: could not convert string to float: '2D1.1 '

## Logistic Regression

In [ ]:
lg = LogisticRegression(random_state = 42)
model_lg = lg.fit(X_train, y_train)

In [ ]:
print(f"Train Score: {lg.score(X_train,y_train)}")
print(f"Test Score: {lg.score(X_test,y_test)}")

## BayesSearchCV using Logistic Regression

In [ ]:
lg = LogisticRegression(random_state =42)


pipe_lg_params = {
    'C': np.logspace(-4, 4, 50), 
    'max_iter': [10_000, 1_000_000],
}   

bs_lg = BayesSearchCV(
    estimator=lg,
    search_spaces=pipe_lg_params,
    n_iter=50, 
    verbose=1,
    cv=5,
    n_jobs=-1
)

In [ ]:
bs_lg.fit(X_train,y_train)

In [ ]:
# Bayes Search
print(f"Train Score: {bs_lg.score(X_train,y_train)}")
print(f"Test Score: {bs_lg.score(X_test,y_test)}")

In [ ]:
y_pred = bs_lg.predict(X_test)

## GridSearchCV using Logistic Regression

In [ ]:
parameters = [{'penalty':['l1','l2']}, 
              {'C':[1, 10, 100, 1000]}]
gs = GridSearchCV(estimator = lg,  
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           verbose=0)

In [ ]:
gs.fit(X_train, y_train) 

In [ ]:
# Bayes Search
print(f"Train Score: {gs.score(X_train,y_train)}")
print(f"Test Score: {gs.score(X_test,y_test)}")

## Random Forest Classifier  

In [ ]:
# instantiate and fit model
rf = RandomForestClassifier(n_estimators=200, 
                            max_depth=None, 
                            max_features='log2', 
                            min_samples_split=50)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
# Bayes Search
print(f"Train Score: {gs.score(X_train,y_train)}")
print(f"Test Score: {gs.score(X_test,y_test)}")

## Export Models

In [ ]:
with open('./sentimp2_bs_lg', 'wb') as pickle_out:
    pickle.dump(bs_lg, pickle_out)

with open('./sentimp2_gs_lg', 'wb') as pickle_out:
    pickle.dump(gs, pickle_out)

with open('./sentimp2_rf', 'wb') as pickle_out:
    pickle.dump(rf, pickle_out)